Download the PDF

In [1]:
!wget https://translation-demo.s3.eu-central-1.amazonaws.com/Data+Mining_+The+Textbook+%5BAggarwal+2015-04-14%5D.pdf

--2024-03-25 16:34:09--  https://translation-demo.s3.eu-central-1.amazonaws.com/Data+Mining_+The+Textbook+%5BAggarwal+2015-04-14%5D.pdf
Resolving translation-demo.s3.eu-central-1.amazonaws.com (translation-demo.s3.eu-central-1.amazonaws.com)... 3.5.136.151, 3.5.134.139, 52.219.169.22, ...
Connecting to translation-demo.s3.eu-central-1.amazonaws.com (translation-demo.s3.eu-central-1.amazonaws.com)|3.5.136.151|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 17227663 (16M) [application/pdf]
Saving to: ‘Data+Mining_+The+Textbook+[Aggarwal+2015-04-14].pdf’

Data+Mining_+The+Te 100%[===================>]  16,43M  9,29MB/s    in 1,8s    

2024-03-25 16:34:11 (9,29 MB/s) - ‘Data+Mining_+The+Textbook+[Aggarwal+2015-04-14].pdf’ saved [17227663/17227663]



Install Poppler (Linux)

In [5]:
!apt-get install -y poppler-utils -q

zsh:1: command not found: apt-get


Install Poppler (Macos)

In [9]:
!brew install poppler

To reinstall 24.03.0, run:
  brew reinstall poppler


In [10]:
!pip install pdf2image -q

In [12]:
from pdf2image import convert_from_path
import os

# Define the PDF file and output folder
pdf_filename = 'Data+Mining_+The+Textbook+[Aggarwal+2015-04-14].pdf'
output_folder = './jpeg'

# Ensure the output directory exists
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Convert the PDF to JPEG images with 150 DPI
images = convert_from_path(pdf_filename, dpi=150)

# Determine the number of digits needed for zero padding
total_pages = len(images)
padding_length = len(str(total_pages))

# Save each page as a JPEG file with zero-padded numbers
for i, page in enumerate(images):
    # Format page number with leading zeros
    page_number = str(i+1).zfill(padding_length)
    output_file_path = os.path.join(output_folder, f"page_{page_number}.jpeg")
    page.save(output_file_path, 'JPEG')

print('PDF conversion completed.')

PDF conversion completed.


Upload to Amazon S3

In [30]:
!pip install python-dotenv -q

In [31]:
from dotenv import load_dotenv
import os
load_dotenv('env.txt')

True

In [32]:
!pip install boto3 -q

In [33]:
import os

# Get the AWS access key and secret from the user
aws_access_key_id = os.environ["AWS_ACCESS_KEY_ID"]
aws_secret_access_key = os.environ["AWS_SECRET_ACCESS_KEY"]

# Build the path to the AWS config file
aws_config_path = os.path.expanduser("~/.aws/config")

# Make sure the directory exists
os.makedirs(os.path.dirname(aws_config_path), exist_ok=True)

# Write the AWS credentials to the config file
with open(aws_config_path, 'w') as config_file:
    config_file.write("[default]\n")
    config_file.write(f"aws_access_key_id={aws_access_key_id}\n")
    config_file.write(f"aws_secret_access_key={aws_secret_access_key}\n")

print(f"Configuration saved to {aws_config_path}")

Configuration saved to /Users/anton/.aws/config


In [1]:
import boto3
import os

# Setup your AWS details
s3_bucket_name = 'translation-demo'
s3_folder_name = 'images'
local_input_dir = './jpeg'

# Initialize AWS client
s3_client = boto3.client('s3')

def upload_files_to_s3(local_input_dir):
    """Uploads local JPEG images to S3 and returns a list of their S3 keys, sorted alphabetically."""
    filenames = [f for f in os.listdir(local_input_dir) if f.endswith('.jpeg')]
    sorted_filenames = sorted(filenames)  # Sort filenames alphabetically
    total_files = len(sorted_filenames)

    s3_keys = []
    for index, filename in enumerate(sorted_filenames, start=1):
        file_path = os.path.join(local_input_dir, filename)
        s3_key = f"{s3_folder_name}/{filename}"
        s3_client.upload_file(file_path, s3_bucket_name, s3_key)
        s3_keys.append(s3_key)
        print(f"Uploading file {index}/{total_files}...")
    return s3_keys


if __name__ == "__main__":
    images = upload_files_to_s3(local_input_dir)
    print("Completed uploading all images. Done uploading.")

Uploading file 1/746...
Uploading file 2/746...
Uploading file 3/746...
Uploading file 4/746...
Uploading file 5/746...
Uploading file 6/746...
Uploading file 7/746...
Uploading file 8/746...
Uploading file 9/746...
Uploading file 10/746...
Uploading file 11/746...
Uploading file 12/746...
Uploading file 13/746...
Uploading file 14/746...
Uploading file 15/746...
Uploading file 16/746...
Uploading file 17/746...
Uploading file 18/746...
Uploading file 19/746...
Uploading file 20/746...
Uploading file 21/746...
Uploading file 22/746...
Uploading file 23/746...
Uploading file 24/746...
Uploading file 25/746...
Uploading file 26/746...
Uploading file 27/746...
Uploading file 28/746...
Uploading file 29/746...
Uploading file 30/746...
Uploading file 31/746...
Uploading file 32/746...
Uploading file 33/746...
Uploading file 34/746...
Uploading file 35/746...
Uploading file 36/746...
Uploading file 37/746...
Uploading file 38/746...
Uploading file 39/746...
Uploading file 40/746...
Uploading

In [ ]:
import boto3
import os
import time

# Setup the output folder
local_output_dir = './output'

# Ensure the output directory exists
if not os.path.exists(local_output_dir):
    os.makedirs(local_output_dir)

# Initialize AWS clients
s3_client = boto3.client('s3')
textract_client = boto3.client('textract')


def get_files_from_s3_folder():
    """Retrieves S3 keys from the specified folder in the S3 bucket."""
    s3_keys = []
    # Use the paginator to handle buckets with many objects
    paginator = s3_client.get_paginator('list_objects_v2')
    page_iterator = paginator.paginate(
        Bucket=s3_bucket_name, Prefix=s3_folder_name + '/')

    for page in page_iterator:
        if 'Contents' in page:
            for obj in page['Contents']:
                s3_key = obj['Key']
                if s3_key.lower().endswith('.jpeg') or s3_key.lower().endswith('.jpg'):
                    s3_keys.append(s3_key)

    # Return sorted list to process in alphabetical order
    return sorted(s3_keys)



def process_image_with_textract(s3_key):
    """Starts a Textract job for a single image and returns the extracted text."""
    # Starting the Textract job
    response = textract_client.start_document_text_detection(
        DocumentLocation={'S3Object': {
            'Bucket': s3_bucket_name, 'Name': s3_key}}
    )
    job_id = response['JobId']
    print(f"Started job {job_id} for {s3_key}")

    # Polling the job status
    while True:
        response = textract_client.get_document_text_detection(JobId=job_id)
        if response['JobStatus'] in ['SUCCEEDED', 'FAILED']:
            break
        time.sleep(5)

    # Processing the results
    text = ''
    if response['JobStatus'] == 'SUCCEEDED':
        for block in response.get('Blocks', []):
            if block['BlockType'] == 'LINE':
                text += block['Text'] + '\n'
    else:
        print(f"Document analysis did not succeed for {s3_key}")
    return text


def save_text_to_file(s3_key, text):
    """Saves extracted text to a .txt file in the local output directory."""
    filename = os.path.basename(s3_key).replace(
        '.jpeg', '.txt').replace('.jpg', '.txt')
    file_path = os.path.join(local_output_dir, filename)
    with open(file_path, 'w', encoding='utf-8') as f:
        f.write(text)
    print(f"Saved text to {file_path}")


images = get_files_from_s3_folder()

for s3_key in images:
    text = process_image_with_textract(s3_key)
    save_text_to_file(s3_key, text)
print("Completed processing all images.")